Using the OSM python bindings to get the geocoding of user location

https://geopy.readthedocs.io/en/stable/#


for GeoJSON analysis 

https://automating-gis-processes.github.io/2016/Lesson3-point-in-polygon.html

Check if point is part of the polygon 


Reading geojson files using geoPandas 

https://www.twilio.com/blog/2017/08/geospatial-analysis-python-geojson-geopandas.html


In [ ]:
!curl --location --request POST 'https://api.placekey.io/v1/placekey' --header 'apikey: {{PLACEKEYAPI}}' --header 'Content-Type: application/json' --data-raw '{ "query": {"query_id": "thisiscustom", "latitude": 37.7371,"longitude": -122.44283}}' 

{"message":"Invalid authentication credentials"}

In [ ]:
!curl --location --request POST 'https://api.placekey.io/v1/placekey' --header 'apikey: PLACEKEYAPI' --header 'Content-Type: application/json' --data-raw '{"query": {"query_id": "thisiscustom", "latitude": 37.7371,"longitude": -122.44283}}'

{"query_id":"thisiscustom","placekey":"@5vg-82n-kzz"}

In [ ]:
!curl --location --request POST 'https://api.placekey.io/v1/placekey' --header 'apikey: PLACEKEYAPI' --header 'Content-Type: application/json' --data-raw '{"query": {"street_address": "1543 Mission Street, Floor 3",  "city": "San Francisco",  "region": "CA",  "postal_code": "94105",  "iso_country_code": "US" }}'

{"query_id":"0","placekey":"22k@5vg-7gq-5mk"}

In [ ]:
!curl --location --request POST 'https://api.placekey.io/v1/placekey' --header 'apikey: PLACEKEYAPI' --header 'Content-Type: application/json' --data-raw '{"query": {"street_address": " ",  "city": "Amsterdam Zuidoost",  "region": " ",  "postal_code": " ",  "iso_country_code": "NLD" }}'

{"message":"parameter query validation failed: parameter iso_country_code validation failed: NLD matches none of the possibilities: us,US"}

In [ ]:
import pandas as pd
dutch_tweets = pd.read_excel('Dutch tweets.xlsx')
municipalities = pd.read_csv('municipalities_v7.csv')

print(dutch_tweets.shape)
print(municipalities.shape)

(1000, 11)
(6840, 26)


In [ ]:
dutch_tweets.head()

,full_text,created_at,screen_name,description,location,Industry,text Translation,Day,Weekday,Month,Year
0,RT @NVALeuven: Onaanvaardbaar! 😡😡😡\n\nSpoedig ...,Mon Sep 14 14:08:15 +0000 2020,karel_mortier,Vader | Echtgenoot | Leuvenaar | Advocaat arb...,NaN,False,RT @NVALeuven: Unacceptable! 😡😡😡\n\nSpeedy rec...,14,0,9,2020
1,Coronabonus voor leiding Zaans Medisch Centrum...,Sat Aug 08 13:55:42 +0000 2020,rightsofearth,Agent provocateur. Stelt rare dingen #stemrech...,Amsterdam Zuidoost,False,Corona Bonus for leadership Zaans Medical Cent...,8,5,8,2020
2,Dubbele verantwoordelijkheid dus voor werkgeve...,Wed Sep 16 09:06:56 +0000 2020,wisbin,𝔻𝕠𝕔𝕖𝕟𝕥 𝕧𝕠/𝕊𝕠𝕔𝕚𝕠𝕝𝕠𝕠𝕘/𝔸𝕟𝕥𝕚𝕥𝕙𝕖ï𝕤𝕥/𝔻𝕒𝕥𝕒 ℝ𝕒𝕡𝕡𝕖𝕣/ℙ𝕚𝕩...,"Schiedam, Nederland",False,Double responsibility So for employers in educ...,16,2,9,2020
3,RT @Elsdijkstra95: Nee tatuurlijk niet he. Waa...,Wed Aug 19 15:27:21 +0000 2020,lorelei3112,Hannelore Bruijn Manus-je-van-alles in apothe...,Streefkerk,False,RT @ Elsdijkstra95 No tatuurlijk not he. Why w...,19,2,8,2020
4,RT @mikercarpenter: News that got completely b...,Wed May 13 14:49:54 +0000 2020,mvanderKist,"Jurist, oud-advocaat, kinderrechten, (jeugd)st...",Apeldoorn e.o. Netherlands,False,RT @mikercarpenter: News that got completely b...,13,2,5,2020


In [ ]:
municipalities.head()

,Unnamed: 0,year,municipality,province,population,latitude,longitude,surface_km2,surface_land_km2,surface_water_km2,university,flag_color1,flag_color2,flag_color3,flag_color4,shield_symbol_1,shield_symbol_2,shield_symbol_3,shieldholder_1,shieldholder_2,shield_crown,asylum_migrants,avg_household_income,avg_woz,avg_ww_ratio,murders
0,1,2018,Aa en Hunze,Drenthe,25243.0,53.010485,6.749528,278.9,276.4,2.5,0,green,yellow,NaN,NaN,crown,cross,NaN,deer,deer,crown,NaN,NaN,NaN,NaN,NaN
1,2,2018,Aalburg,Noord-Brabant,12859.0,51.751294,5.057085,53.2,50.4,2.8,0,black,red,yellow,NaN,wheel,lion,NaN,NaN,NaN,crown,NaN,NaN,NaN,NaN,NaN
2,3,2018,Aalsmeer,Noord-Holland,30792.0,52.262164,4.761922,32.3,20.4,11.9,0,red,green,black,NaN,lion,eel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2018,Aalten,Gelderland,27030.0,51.926667,6.580678,97.0,96.5,0.5,0,white,green,NaN,NaN,tree,sword,NaN,NaN,NaN,crown,NaN,NaN,NaN,NaN,NaN
4,5,2018,Achtkarspelen,Friesland,28002.0,53.210357,6.153565,104.0,102.4,1.6,0,white,red,green,blue,church,NaN,NaN,NaN,NaN,crown,NaN,NaN,NaN,NaN,NaN


In [ ]:
municipalities[municipalities['municipality'] == 'Streefkerk']

,Unnamed: 0,year,municipality,province,population,latitude,longitude,surface_km2,surface_land_km2,surface_water_km2,university,flag_color1,flag_color2,flag_color3,flag_color4,shield_symbol_1,shield_symbol_2,shield_symbol_3,shieldholder_1,shieldholder_2,shield_crown,asylum_migrants,avg_household_income,avg_woz,avg_ww_ratio,murders


In [ ]:
!pip install geopandas
!pip install geopy


In [ ]:
import geopy.geocoders
from geopy.geocoders import Nominatim

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

In [ ]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 48.85614465, Longitude = 2.297820393322227


In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# 2- - create location column
dutch_tweets['point_info'] = dutch_tweets['location'].apply(geocode)

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
dutch_tweets['point'] = dutch_tweets['point_info'].apply(lambda loc: tuple(loc.point) if loc else None)



In [ ]:
# 4 - split point column into latitude, longitude and altitude columns
dutch_tweets[['latitude', 'longitude', 'altitude']] = pd.DataFrame(dutch_tweets['point'].tolist(), index=dutch_tweets.index)

In [ ]:
dutch_tweets.head()

,full_text,created_at,screen_name,description,location,Industry,text Translation,Day,Weekday,Month,Year,point_info,point,latitude,longitude,altitude
0,RT @NVALeuven: Onaanvaardbaar! 😡😡😡\n\nSpoedig ...,Mon Sep 14 14:08:15 +0000 2020,karel_mortier,Vader | Echtgenoot | Leuvenaar | Advocaat arb...,NaN,False,RT @NVALeuven: Unacceptable! 😡😡😡\n\nSpeedy rec...,14,0,9,2020,,"(46.3144754, 11.0480288, 0.0)",46.314475,11.048029,0.0
1,Coronabonus voor leiding Zaans Medisch Centrum...,Sat Aug 08 13:55:42 +0000 2020,rightsofearth,Agent provocateur. Stelt rare dingen #stemrech...,Amsterdam Zuidoost,False,Corona Bonus for leadership Zaans Medical Cent...,8,5,8,2020,"(Amsterdam-Zuidoost, Golfbaanpad, Holendrecht,...","(52.2835706, 4.9524489, 0.0)",52.283571,4.952449,0.0
2,Dubbele verantwoordelijkheid dus voor werkgeve...,Wed Sep 16 09:06:56 +0000 2020,wisbin,𝔻𝕠𝕔𝕖𝕟𝕥 𝕧𝕠/𝕊𝕠𝕔𝕚𝕠𝕝𝕠𝕠𝕘/𝔸𝕟𝕥𝕚𝕥𝕙𝕖ï𝕤𝕥/𝔻𝕒𝕥𝕒 ℝ𝕒𝕡𝕡𝕖𝕣/ℙ𝕚𝕩...,"Schiedam, Nederland",False,Double responsibility So for employers in educ...,16,2,9,2020,"(Schiedam, Zuid-Holland, Nederland, (51.930045...","(51.9300454, 4.386451052615731, 0.0)",51.930045,4.386451,0.0
3,RT @Elsdijkstra95: Nee tatuurlijk niet he. Waa...,Wed Aug 19 15:27:21 +0000 2020,lorelei3112,Hannelore Bruijn Manus-je-van-alles in apothe...,Streefkerk,False,RT @ Elsdijkstra95 No tatuurlijk not he. Why w...,19,2,8,2020,"(Streefkerk, Molenlanden, Zuid-Holland, Nederl...","(51.902614650000004, 4.775125690329026, 0.0)",51.902615,4.775126,0.0
4,RT @mikercarpenter: News that got completely b...,Wed May 13 14:49:54 +0000 2020,mvanderKist,"Jurist, oud-advocaat, kinderrechten, (jeugd)st...",Apeldoorn e.o. Netherlands,False,RT @mikercarpenter: News that got completely b...,13,2,5,2020,"(Apeldoorn, Gelderland, Nederland, (52.2156457...","(52.2156457, 5.9639485, 0.0)",52.215646,5.963948,0.0


In [ ]:

dutch_tweets.to_excel('dutch_tweets.xlsx', index=False)

In [ ]:
# If dutch_tweet['location'] was empty or NaN then remove the point information

dutch_tweets.loc[dutch_tweets['location'].isnull(), ['point_info', 'point', 'latitude', 'longitude', 'altitude']] = ""

In [ ]:
dutch_tweets[['latitude', 'longitude', 'altitude']] = pd.DataFrame(dutch_tweets['point'].tolist(), index=dutch_tweets.index)

In [ ]:
dutch_tweets.head()

,full_text,created_at,screen_name,description,location,Industry,text Translation,Day,Weekday,Month,Year,point_info,point,latitude,longitude,altitude
0,RT @NVALeuven: Onaanvaardbaar! 😡😡😡\n\nSpoedig ...,Mon Sep 14 14:08:15 +0000 2020,karel_mortier,Vader | Echtgenoot | Leuvenaar | Advocaat arb...,NaN,False,RT @NVALeuven: Unacceptable! 😡😡😡\n\nSpeedy rec...,14,0,9,2020,,,,,
1,Coronabonus voor leiding Zaans Medisch Centrum...,Sat Aug 08 13:55:42 +0000 2020,rightsofearth,Agent provocateur. Stelt rare dingen #stemrech...,Amsterdam Zuidoost,False,Corona Bonus for leadership Zaans Medical Cent...,8,5,8,2020,"(Amsterdam-Zuidoost, Golfbaanpad, Holendrecht,...","(52.2835706, 4.9524489, 0.0)",52.2836,4.95245,0
2,Dubbele verantwoordelijkheid dus voor werkgeve...,Wed Sep 16 09:06:56 +0000 2020,wisbin,𝔻𝕠𝕔𝕖𝕟𝕥 𝕧𝕠/𝕊𝕠𝕔𝕚𝕠𝕝𝕠𝕠𝕘/𝔸𝕟𝕥𝕚𝕥𝕙𝕖ï𝕤𝕥/𝔻𝕒𝕥𝕒 ℝ𝕒𝕡𝕡𝕖𝕣/ℙ𝕚𝕩...,"Schiedam, Nederland",False,Double responsibility So for employers in educ...,16,2,9,2020,"(Schiedam, Zuid-Holland, Nederland, (51.930045...","(51.9300454, 4.386451052615731, 0.0)",51.93,4.38645,0
3,RT @Elsdijkstra95: Nee tatuurlijk niet he. Waa...,Wed Aug 19 15:27:21 +0000 2020,lorelei3112,Hannelore Bruijn Manus-je-van-alles in apothe...,Streefkerk,False,RT @ Elsdijkstra95 No tatuurlijk not he. Why w...,19,2,8,2020,"(Streefkerk, Molenlanden, Zuid-Holland, Nederl...","(51.902614650000004, 4.775125690329026, 0.0)",51.9026,4.77513,0
4,RT @mikercarpenter: News that got completely b...,Wed May 13 14:49:54 +0000 2020,mvanderKist,"Jurist, oud-advocaat, kinderrechten, (jeugd)st...",Apeldoorn e.o. Netherlands,False,RT @mikercarpenter: News that got completely b...,13,2,5,2020,"(Apeldoorn, Gelderland, Nederland, (52.2156457...","(52.2156457, 5.9639485, 0.0)",52.2156,5.96395,0


In [ ]:
dutch_tweets.to_excel('dutch_tweets.xlsx', index=False)

Source:

https://www.twilio.com/blog/2017/08/geospatial-analysis-python-geojson-geopandas.html

In [ ]:
import geopandas as gpd

provinces = gpd.read_file('provinces.geojson')

print(provinces.shape)
provinces.head()

(12, 4)


,name,regioFacetId,level,geometry
0,Drenthe,tcm:106-353397-1024,3,"POLYGON ((6.41328 52.98552, 6.36252 53.03397, ..."
1,Flevoland,tcm:106-353410-1024,3,"POLYGON ((5.36115 52.67573, 5.37726 52.76481, ..."
2,Friesland (Fryslân),tcm:106-353417-1024,3,"POLYGON ((5.08707 53.32307, 5.10178 53.36803, ..."
3,Gelderland,tcm:106-353445-1024,3,"POLYGON ((5.60602 51.99416, 5.59200 52.00139, ..."
4,Groningen,tcm:106-353502-1024,3,"POLYGON ((6.28698 53.34138, 6.27368 53.34527, ..."


In [ ]:
from shapely.geometry import Point, Polygon

p1 = Point(dutch_tweets['point'][1])


In [ ]:
provinces['geometry'][0].contains(p1)

False

In [ ]:
def checkProvince(p1):
  #print(len(p1))
  if p1 != None and len(p1) >= 2 :
    p1 = Point(p1[1], p1[0])  
    provinces['InProvince'] = provinces['geometry'].apply(lambda x: x.contains(p1))
    selProvince = provinces[provinces['InProvince'] == True]
    #print(selProvince)
    if selProvince.shape[0] > 0:
      return selProvince['name'].values[0]
    else:
      return False
  else:
    return False

In [ ]:
checkProvince(dutch_tweets['point'][1])

'Noord-Holland'

In [ ]:
dutch_tweets['Province'] = dutch_tweets['point'].apply(lambda x: checkProvince(x))

In [ ]:
dutch_tweets.head()

,full_text,created_at,screen_name,description,location,Industry,text Translation,Day,Weekday,Month,Year,point_info,point,latitude,longitude,altitude,Province
0,RT @NVALeuven: Onaanvaardbaar! 😡😡😡\n\nSpoedig ...,Mon Sep 14 14:08:15 +0000 2020,karel_mortier,Vader | Echtgenoot | Leuvenaar | Advocaat arb...,NaN,False,RT @NVALeuven: Unacceptable! 😡😡😡\n\nSpeedy rec...,14,0,9,2020,,,,,,False
1,Coronabonus voor leiding Zaans Medisch Centrum...,Sat Aug 08 13:55:42 +0000 2020,rightsofearth,Agent provocateur. Stelt rare dingen #stemrech...,Amsterdam Zuidoost,False,Corona Bonus for leadership Zaans Medical Cent...,8,5,8,2020,"(Amsterdam-Zuidoost, Golfbaanpad, Holendrecht,...","(52.2835706, 4.9524489, 0.0)",52.2836,4.95245,0,Noord-Holland
2,Dubbele verantwoordelijkheid dus voor werkgeve...,Wed Sep 16 09:06:56 +0000 2020,wisbin,𝔻𝕠𝕔𝕖𝕟𝕥 𝕧𝕠/𝕊𝕠𝕔𝕚𝕠𝕝𝕠𝕠𝕘/𝔸𝕟𝕥𝕚𝕥𝕙𝕖ï𝕤𝕥/𝔻𝕒𝕥𝕒 ℝ𝕒𝕡𝕡𝕖𝕣/ℙ𝕚𝕩...,"Schiedam, Nederland",False,Double responsibility So for employers in educ...,16,2,9,2020,"(Schiedam, Zuid-Holland, Nederland, (51.930045...","(51.9300454, 4.386451052615731, 0.0)",51.93,4.38645,0,Zuid-Holland
3,RT @Elsdijkstra95: Nee tatuurlijk niet he. Waa...,Wed Aug 19 15:27:21 +0000 2020,lorelei3112,Hannelore Bruijn Manus-je-van-alles in apothe...,Streefkerk,False,RT @ Elsdijkstra95 No tatuurlijk not he. Why w...,19,2,8,2020,"(Streefkerk, Molenlanden, Zuid-Holland, Nederl...","(51.902614650000004, 4.775125690329026, 0.0)",51.9026,4.77513,0,Zuid-Holland
4,RT @mikercarpenter: News that got completely b...,Wed May 13 14:49:54 +0000 2020,mvanderKist,"Jurist, oud-advocaat, kinderrechten, (jeugd)st...",Apeldoorn e.o. Netherlands,False,RT @mikercarpenter: News that got completely b...,13,2,5,2020,"(Apeldoorn, Gelderland, Nederland, (52.2156457...","(52.2156457, 5.9639485, 0.0)",52.2156,5.96395,0,Gelderland


In [ ]:
dutch_tweets.to_excel('dutch_tweets.xlsx', index=False)